In [ ]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import joblib
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

In [ ]:
#load cleaned data
df = pd.read_json(
    'G:/Shared drives\SIADS - 697 Capstone Team Drive/datasets/cleaned_data.json',
)

In [ ]:
# load category string index
with open('category_string_index.pickle', 'rb') as filehandle:
    category_string_index = pickle.load(filehandle)

In [ ]:
df.info()

In [ ]:
def get_index(x):
    try:
        return(category_string_index[x])
    except:
        return(np.nan)

In [ ]:
df['subcategory_index'] = df['category_string'].apply(get_index)

In [ ]:
df['subcategory_index'].isnull().sum()

In [ ]:
#dropping NA to make progress
df.dropna(inplace=True)
df.info()

In [ ]:
def create_M1(df, features = '4_features_combined', components = 10, ngrams = 2):
    # create vectorizer title and category plus description
    vectorizer= TfidfVectorizer(stop_words="english", max_df = 0.95, ngram_range = (1,ngrams), sublinear_tf = True)
    X = vectorizer.fit_transform(df[features])
    #create LSI model
    lsi=TruncatedSVD(n_components=components)  
    lsi_text_transformed = lsi.fit_transform(X)
    
    return lsi_text_transformed

In [ ]:
### create machine learning models

In [ ]:
# create y target values and initialize dataframe to store evaluation results
y = df['subcategory_index'].astype('int')
summary = pd.DataFrame(columns = ['features', 'components', 'n_gram', 'accuracy', 'macro_f1', 'micro_f1', 'weighted_f1'])

In [ ]:
def create_eval_Kneighbor(feat, compon, grams, y):
    X = create_M1(df, features = feat, components = compon, ngrams = grams)
    #split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    #train the data
    NN = KNeighborsClassifier(n_neighbors=5)
    NN.fit(X_train, y_train)
    # metrics
    pred = NN.predict(X_test)
    accuracy = accuracy_score(y_test,pred)
    macro_f1 =  f1_score(y_test, pred, average='macro')
    micro_f1 = f1_score(y_test, pred, average='micro')
    weighted_f1 = f1_score(y_test, pred, average='weighted')
    summary.loc[len(summary.index)] = [feat, compon , grams, accuracy, macro_f1, micro_f1, weighted_f1]

In [ ]:

for ngram in [1,2,3]:
    for features in ['4_features_combined','5_features_combined']:
        for comp in [50]:
            print(ngram, features, comp)
            create_eval_Kneighbor(features, comp, ngram, y)

In [ ]:
summary.to_csv('summary_knn_m1_metrics_v4.csv', index=False)